In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from linearmodels.iv import IV2SLS  
from statsmodels.formula.api import ols
from scipy.optimize import minimize_scalar

# データの準備（第２回と同じコード）

In [18]:
# 自動車データ 
data = pd.read_csv("source/CleanData_20180222.csv", encoding='shift_jis')

# 潜在的な市場規模としての、家計数データ（出所：https://www.airia.or.jp/publish/file/r5c6pv0000006s9v-att/r5c6pv0000006saa.pdf）
dataHH = pd.read_csv("source/HHsize.csv")

# 価格の実質化のための、消費者物価指数（出所：https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031431770&fileKind=1）
dataCPI = pd.read_csv("source/zni2015s.csv", encoding='shift_jis')

# 必要な列のみ抽出・列名の変更
dataCPI = dataCPI[6:56]
dataCPI = dataCPI.rename(columns={'類・品目':'year', '総合':'cpi'})

# 型の変更
dataCPI = dataCPI.astype({'year': 'f2', 'cpi': 'f2'})

# 必要な変数のみ抽出
data = data.loc[:, ['Maker', 'Name', 'Year', 'Sales', 'Model', 'price', 'kata', 'weight', 
                   'FuelEfficiency', 'HorsePower', 'overall_length', 'overall_width','overall_height']]
# 後ほどのマージのために列名の変更
data = data.rename(columns={'Year' : 'year'}) 

# データのマージ
data = pd.merge(data, dataHH,how='left')
data = pd.merge(data, dataCPI,how='left')

# 燃費が欠損しているデータを削除する
#data = data['FuelEfficiency'].dropna()

data = data.dropna()

# 2016年を基準として、価格の実質化を実行する。
# 価格の単位は100万円。

filtered_dataCPI = dataCPI[dataCPI['year'] == 2016]

# 'cpi' 列のみを抽出
cpi2016 = filtered_dataCPI['cpi']

cpi2016 = cpi2016.astype('float64')
data['cpi'] = data['cpi'].astype('float64')
data['price'] = data['price'].astype('float64')

# 'price' 列を 'cpi' 列と 'cpi2016' 列を用いて変換
# data['cpi']/cpi2016が全てNaNとなり、解決できなかったためコメントアウト。 
# 2023/9/16追記：float64に変換することで成功。
# data['price'] = data['price'] / (data['cpi'] / cpi2016) 

# 'price' 列を100で割る
data['price'] = data['price'] / 100

# 'cpi' 列を削除
data = data.drop(columns=['cpi'])

# サイズ(高さ＊幅＊長さ)、燃費の重量に対する比率を定義する。
# 'size' 列を追加
data = data.assign(size=lambda x: (x['overall_length'] / 1000) * (x['overall_width'] / 1000) * (x['overall_height'] / 1000))

# 'hppw' 列を追加
data = data.assign(hppw=lambda x: x['HorsePower'] / x['weight'])

# 'HorsePower' と 'weight' 列を削除
data = data.drop(columns=['HorsePower', 'weight'])

# 列名が 'overall' で始まる列を削除
data = data.drop(columns=data.filter(like='overall').columns)

# 自動車の車種IDを作成する。
grouped_data = data.groupby('Name')
data['NameID'] = grouped_data.ngroup()
data = data.reset_index(drop=True)

# 列の順序を変更 (NameID を year の前に移動)
data = data[['NameID', 'year'] + [col for col in data.columns if col not in ['NameID', 'year']]]

# data['HH']から、コンマを消去、int型に変換する。
data['HH'] = data['HH'].str.replace(',', '').astype(int)

# Market ShareとOutside option shareを定義する。
grouped_data = data.groupby('year')
data['inside_total'] = grouped_data['Sales'].transform('sum')

data = data.assign(outside_total = lambda x: x['HH']-x['inside_total'])
data = data.assign(share = data['Sales']/data['HH'])
data = data.assign(share0 = data['outside_total']/data['HH'])
data = data.drop(columns=['inside_total', 'outside_total'])

# 操作変数の構築

In [19]:
# マーケット・企業レベル（year・Maker）における、各製品属性の和と自乗和を計算する。
data = data.groupby(['year', 'Maker']).apply(lambda x: x.assign(
    hppw_sum_own=x['hppw'].sum(),
    FuelEfficiency_sum_own=x['FuelEfficiency'].sum(),
    size_sum_own=x['size'].sum(),
    hppw_sqr_sum_own=(x['hppw'] ** 2).sum(),
    FuelEfficiency_sqr_sum_own=(x['FuelEfficiency'] ** 2).sum(),
    size_sqr_sum_own=(x['size'] ** 2).sum(),
    group_n = len(x)
)).reset_index(drop=True)

# グループ化を解除
data = data.reset_index(drop=True)

# マーケットレベル（year）での、各製品属性の和を計算。
data = data.groupby('year').apply(lambda x: x.assign(
    hppw_sum_mkt=x['hppw'].sum(),
    FuelEfficiency_sum_mkt=x['FuelEfficiency'].sum(),
    size_sum_mkt=x['size'].sum(),
    hppw_sqr_sum_mkt=(x['hppw'] ** 2).sum(),
    FuelEfficiency_sqr_sum_mkt=(x['FuelEfficiency'] ** 2).sum(),
    size_sqr_sum_mkt=(x['size'] ** 2).sum(),
    mkt_n=len(x)
)).reset_index(drop=True)

# グループ化を解除
data = data.reset_index(drop=True)

# BLP操作変数の構築

data = data.assign(iv_BLP_own_hppw = lambda x: x['hppw_sum_own']-x['hppw'])
data = data.assign(iv_BLP_own_FuelEfficiency = lambda x: x['FuelEfficiency_sum_own']-x['FuelEfficiency'])
data = data.assign(iv_BLP_own_size = lambda x: x['size_sum_own']-x['size'])
data = data.assign(iv_BLP_other_hppw = lambda x: x['hppw_sum_mkt']-x['hppw_sum_own'])
data = data.assign(iv_BLP_other_FuelEfficiency = lambda x: x['FuelEfficiency_sum_mkt']-x['FuelEfficiency_sum_own'])
data = data.assign(iv_BLP_other_size = lambda x: x['size_sum_mkt']-x['size_sum_own'])

# Differentiation IVの構築
data = data.assign(iv_GH_own_hppw = lambda x: (x['group_n']-1) * x['hppw']**2 + 
                   (x['hppw_sqr_sum_own']-x['hppw']**2) 
                   - 2*x['hppw']*(x['hppw_sum_own']-x['hppw']))

data = data.assign(iv_GH_own_FuelEfficiency = lambda x: (x['group_n']-1) * x['FuelEfficiency']**2 + 
                   (x['FuelEfficiency_sqr_sum_own']-x['FuelEfficiency']**2) 
                   - 2*x['FuelEfficiency']*(x['FuelEfficiency_sum_own']-x['FuelEfficiency']))

data = data.assign(iv_GH_own_size = lambda x: (x['group_n']-1) * x['size']**2 + 
                   (x['size_sqr_sum_own']-x['size']**2) 
                   - 2*x['size']*(x['size_sum_own']-x['size']))

data = data.assign(iv_GH_other_hppw = lambda x: (x['mkt_n']-x['group_n']) * x['hppw']**2 + 
                   (x['hppw_sqr_sum_mkt']-x['hppw_sqr_sum_own']) 
                   - 2*x['hppw']*(x['hppw_sum_mkt']-x['hppw_sum_own']))

data = data.assign(iv_GH_other_FuelEfficiency = lambda x: (x['mkt_n']-x['group_n']) * x['FuelEfficiency']**2 + 
                   (x['FuelEfficiency_sqr_sum_mkt']-x['FuelEfficiency_sqr_sum_own']) 
                   - 2*x['FuelEfficiency']*(x['FuelEfficiency_sum_mkt']-x['FuelEfficiency_sum_own']))

data = data.assign(iv_GH_other_size = lambda x: (x['mkt_n']-x['group_n']) * x['size']**2 + 
                   (x['size_sqr_sum_mkt']-x['size_sqr_sum_own']) 
                   - 2*x['size']*(x['size_sum_mkt']-x['size_sum_own']))

# 不要な列を削除する
columns_to_remove = [col for col in data.columns if (
    col.startswith("sum_own") or
    col.startswith("sum_mkt") or
    col.startswith("sqr_sum_own") or
    col.startswith("sqr_sum_mkt") or
    col == "mkt_n" or
    col == "group_n"
)]
data = data.drop(columns=columns_to_remove)

# Logit Modelにおける価格弾力性
## 日評自動車のデータセットを用意

In [44]:
# 車種のID
IDvec = sorted(data['NameID'].unique())
J = len(IDvec)

# 乱数のseedを固定
# PythonとRでは乱数のseedを同じにしても同じ結果はでてこない
np.random.seed(125)
# IDvecから30個のサンプルを非復元抽出して取得
NIPPYOautoIDvec = np.random.choice(IDvec, size=30, replace=False)

# サンプルを昇順にソート
NIPPYOautoIDvec.sort()

# NIPPYOautoIDvecに含まれるIDを持つ行を抽出
data_NIPPYO = data[data['NameID'].isin(NIPPYOautoIDvec)]



# 必要な列を選択
data_NIPPYO = data_NIPPYO[['Sales', 'price', 'hppw', 'FuelEfficiency', 'size', 'year', 'NameID', 'share']]
# 列 'Sales' と 'price' の対数を計算し、新しい列に追加
data_NIPPYO['log_sales'] = np.log(data_NIPPYO['Sales'])
data_NIPPYO['log_price'] = np.log(data_NIPPYO['price'])

## Logit Modelの価格弾力性行列を作成する
- Logit Modelにおける需要の価格弾力性 $\eta_{jlt}$ は、以下の通り。
$$ \begin{align}
\eta_{jlt} = \frac{\partial q_{jt}}{\partial p_{lt}}\frac{p_{lt}}{q_{jt}} = \left\{
\begin{array}{ll}
-\alpha p_{jt}(1-s_{jt}) & l = j \\
\alpha p_{lt}s_{lt} & l \ne j
\end{array}
\right.
\end{align} $$ 
- 財 $l$ の価格が上昇した時の他の財の需要の変化率は、すべての 財　$j \ne l$ で等しくなる。
- Logit Modelで得られるマーケットシェアは、選択確率と等しくなるため以下の通り。

$$
\begin{align}
s_{jt}  =\left\{
\begin{array}{ll}
\frac{\exp(\delta_{jt})}{1+\sum_{l=1}^{J_t}\exp(\delta_{jt})} & j=1,\cdots, J_t\\
\frac{1}{1+\sum_{l=1}^{J_t}\exp(\delta_{jt})} & j=0
\end{array}
\right.
\end{align}
$$

- よって、任意の2財、$j$と$l$のマーケットシェアの比率は以下の通り。
$$
\frac{s_j}{s_l} = \frac{\exp(\delta_j)}{\exp(\delta_l)}
$$

- これは、ある財$j$と財$l$を選ぶ比率は、他の選択可能な製品や製品属性とは無関係になることを意味している。
- これを無関係な選択肢からの独立性（independence of irrelevant alternatives: **IIA**）が成立しているという。
- 以下では  IIAが成立していることを確かめる。

In [45]:
data['logit_share'] = np.log(data['share']) - np.log(data['share0'])
# Differentiation IVを用いた結果
formula = 'logit_share ~1+ hppw + FuelEfficiency + size + [price ~ iv_GH_own_hppw + iv_GH_own_FuelEfficiency + iv_GH_own_size +iv_GH_other_hppw + iv_GH_other_FuelEfficiency + iv_GH_other_size]'
mod_1 = IV2SLS.from_formula(formula, data=data)
iv_GH = mod_1.fit(cov_type='unadjusted')

In [47]:
dt2016 = data_NIPPYO[data_NIPPYO['year'] == 2016].sort_values('NameID')
price = dt2016['price']
share = dt2016['share']
NameID = dt2016['NameID']

# 自己弾力性
own_elas = 